Dependencies
---------------------

In [6]:
!pip install librosa
!pip install umap-learn
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 52.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.0/401.0 kB 56.1 MB/s eta 0:00:00


Algorithm and report
-----------------------

In [9]:
# pca_kmeans_pipeline.py  — TEST-ONLY version

import os
import numpy as np
import librosa
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import (
    adjusted_rand_score, adjusted_mutual_info_score,
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
)
from scipy.optimize import linear_sum_assignment
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any, Optional, Sequence

# Safe UMAP import
import umap.umap_ as umap
import plotly.express as px

# ----------------------------
#  Hungarian Accuracy
# ----------------------------
def hungarian_accuracy(y_true, y_pred):
    D = int(max(y_pred.max(), y_true.max())) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(len(y_pred)):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    return float(sum(w[i, j] for i, j in zip(row_ind, col_ind)) / len(y_pred))

# ----------------------------
#  Helpers to get paths/labels and load features for a SUBSET ONLY
# ----------------------------
def _sorted_paths_and_labels(audio_dir: str) -> Tuple[list, np.ndarray]:
    paths = sorted(
        [os.path.join(audio_dir, f) for f in os.listdir(audio_dir)
         if f.endswith(".npy") and f != "labels.npy"],
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )
    labels = np.load(os.path.join(audio_dir, "labels.npy"))
    if len(paths) != len(labels):
        raise ValueError(f"{len(paths)} audio files != {len(labels)} labels")
    return paths, labels

def _load_logmel_subset(
    paths: Sequence[str],
    labels: np.ndarray,
    indices: Sequence[int],
    sr: int = 16000,
    n_mels: int = 64
) -> Tuple[np.ndarray, np.ndarray]:
    X, y = [], []
    for idx in indices:
        p = paths[idx]
        try:
            audio = np.load(p).astype(np.float32)
            m = np.max(np.abs(audio))
            if m > 0:
                audio = audio / m
            mel = librosa.feature.melspectrogram(
                y=audio, sr=sr, n_fft=1024, hop_length=512, n_mels=n_mels
            )
            logmel = librosa.power_to_db(mel + 1e-8)
            X.append(logmel.flatten())
            y.append(labels[idx])
        except Exception as e:
            print(f"Skipping {p}: {e}")
    return np.asarray(X), np.asarray(y)

# ----------------------------
#  Internal cluster indices (on TEST space)
# ----------------------------
def _internal_indices(X: np.ndarray, labels: np.ndarray) -> Dict[str, float]:
    out = {"silhouette": float("nan"),
           "davies_bouldin": float("nan"),
           "calinski_harabasz": float("nan")}
    uniq = np.unique(labels)
    if len(uniq) <= 1 or len(uniq) >= len(labels):
        return out
    try: out["silhouette"] = float(silhouette_score(X, labels))
    except Exception: pass
    try: out["davies_bouldin"] = float(davies_bouldin_score(X, labels))
    except Exception: pass
    try: out["calinski_harabasz"] = float(calinski_harabasz_score(X, labels))
    except Exception: pass
    return out

# ----------------------------
#  Runner (TEST-ONLY)
# ----------------------------
def run_pca_kmeans_test_only(
    dataset_path: str,
    *,
    n_components: int = 100,
    random_state: int = 42,
    do_umap: bool = True,
    umap_components: int = 3,
    umap_html_path: str = "pca_kmeans_umap.html",
    kmeans_k: Optional[int] = None,     # choose k explicitly
    test_size: float = 0.2,             # 20% test set
    test_indices: Optional[Sequence[int]] = None,  # pass a fixed 20% if you want consistency across methods
) -> Dict[str, Any]:
    """
    TEST-ONLY pipeline:
      - Make a stratified 80/20 split by indices (or use provided test_indices)
      - Load + process features for the TEST split ONLY
      - Fit StandardScaler/PCA on TEST only
      - Fit KMeans on TEST only, compute ARI/AMI/Hungarian + Silhouette/DB/CH
      - (Optional) UMAP of TEST embeddings
    """
    label_path = os.path.join(dataset_path, "labels.npy")
    if not os.path.isdir(dataset_path):
        raise FileNotFoundError(f"Dataset directory not found: {dataset_path}")
    if not os.path.isfile(label_path):
        raise FileNotFoundError(f"Label file not found: {label_path}")

    # 1) Fixed split by indices
    paths, labels = _sorted_paths_and_labels(dataset_path)
    all_idx = np.arange(len(paths))
    if test_indices is None:
        # Stratified split; we ignore the train indices and do not process/train on them.
        _, test_idx = train_test_split(
            all_idx, test_size=test_size, stratify=labels, random_state=random_state
        )
    else:
        test_idx = np.asarray(list(test_indices), dtype=int)

    # 2) Load ONLY the TEST subset
    X_test, y_test = _load_logmel_subset(paths, labels, test_idx)
    if X_test.size == 0:
        raise ValueError("No test features were loaded.")

    # 3) Scale + PCA (fit on TEST ONLY)
    scaler = StandardScaler().fit(X_test)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=n_components, random_state=random_state).fit(X_test_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    # 4) KMeans on TEST ONLY
    n_clusters = int(kmeans_k) if kmeans_k is not None else len(np.unique(y_test))
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=random_state)
    test_preds = kmeans.fit_predict(X_test_pca)

    # 5) Metrics (external + internal) on TEST
    ari  = adjusted_rand_score(y_test, test_preds)
    ami  = adjusted_mutual_info_score(y_test, test_preds)
    hacc = hungarian_accuracy(y_test, test_preds)
    extra = _internal_indices(X_test_pca, test_preds)

    print(f"\n📊 TEST-ONLY PCA+KMeans (k={n_clusters})")
    print(f"ARI: {ari:.4f} | AMI: {ami:.4f} | Hungarian: {hacc:.4f}")
    print(f"Silhouette: {extra['silhouette']:.4f} | "
          f"Davies–Bouldin: {extra['davies_bouldin']:.4f} | "
          f"Calinski–Harabasz: {extra['calinski_harabasz']:.2f}")

    results: Dict[str, Any] = {
        "test_samples": int(len(X_test)),
        "n_components": int(n_components),
        "kmeans_k": int(n_clusters),
        "metrics": {
            "ari": float(ari),
            "ami": float(ami),
            "hungarian_accuracy": float(hacc),
            "silhouette": float(extra["silhouette"]),
            "davies_bouldin": float(extra["davies_bouldin"]),
            "calinski_harabasz": float(extra["calinski_harabasz"]),
        },
        "used_test_indices": np.asarray(test_idx).tolist(),
    }

    # 6) Optional UMAP (TEST ONLY)
    if do_umap:
        if umap_components not in (2, 3):
            raise ValueError("umap_components must be 2 or 3")
        reducer = umap.UMAP(n_components=umap_components, random_state=random_state)
        X_umap = reducer.fit_transform(X_test_pca)
        if umap_components == 3:
            fig = px.scatter_3d(
                x=X_umap[:, 0], y=X_umap[:, 1], z=X_umap[:, 2],
                color=y_test.astype(str),
                title=(f"TEST-ONLY PCA+KMeans (k={n_clusters}) | "
                       f"ARI {ari:.4f}, AMI {ami:.4f}, H-Acc {hacc:.4f}"),
                labels={"x": "UMAP-1", "y": "UMAP-2", "z": "UMAP-3"},
                opacity=0.7
            )
        else:
            fig = px.scatter(
                x=X_umap[:, 0], y=X_umap[:, 1],
                color=y_test.astype(str),
                title=(f"TEST-ONLY PCA+KMeans (k={n_clusters}) | "
                       f"ARI {ari:.4f}, AMI {ami:.4f}"),
                labels={"x": "UMAP-1", "y": "UMAP-2"},
                opacity=0.7
            )
        fig.write_html(umap_html_path)
        print(f"Saved UMAP visualization to '{umap_html_path}'")
        results["umap_html_path"] = umap_html_path

    return results

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
PCA + KMeans (TEST-ONLY) — Unsupervised Clustering HTML Report

- Accepts one or more dataset directories and matching location tags
- Makes a stratified 80/20 split (or you can pass fixed TEST indices per dataset)
- Loads ONLY the TEST portions, computes log-mel (if waveform) or uses given 2D specs
- Fits StandardScaler+PCA on the combined TEST features
- Runs KMeans on TEST embeddings and reports:
    * Global internal metrics: Silhouette (↑), DBI (↓), CH (↑)
    * External metrics vs labels.npy: ARI, AMI, Hungarian
- UMAP 3D scatter (TEST embeddings)
- Per-cluster cards: size, variance, embedding cosine, location entropy, quality, novelty
- Spectrogram thumbnails per cluster (nearest-to-center)
- Signal-space consistency per cluster (mean spectrogram cosine; optional DTW)

Requires: numpy, librosa, scikit-learn, plotly, matplotlib, umap-learn
Optional: fastdtw  (for DTW; otherwise shows NaN)
"""

import os, math, base64, time
from io import BytesIO
from collections import Counter
from typing import List, Optional, Dict, Tuple, Sequence

import numpy as np
import librosa
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ML / metrics
import umap.umap_ as umap
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score, adjusted_mutual_info_score
)
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import linear_sum_assignment
from scipy.signal import spectrogram, get_window
from scipy.stats import entropy


# ========================= Helpers: files, labels, audio =========================

def _sorted_paths_and_labels(audio_dir: str) -> Tuple[List[str], np.ndarray]:
    paths = sorted(
        [os.path.join(audio_dir, f) for f in os.listdir(audio_dir)
         if f.endswith(".npy") and f != "labels.npy"],
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )
    lbl_path = os.path.join(audio_dir, "labels.npy")
    if not os.path.isfile(lbl_path):
        raise FileNotFoundError(f"Label file not found: {lbl_path}")
    labels = np.load(lbl_path, allow_pickle=True)
    if len(paths) != len(labels):
        raise ValueError(f"{len(paths)} audio files != {len(labels)} labels in {audio_dir}")
    return paths, labels


def _load_logmel_or_flatten(arr: np.ndarray, *, sr: int = 16000, n_mels: int = 64) -> np.ndarray:
    """If 1D waveform: compute log-mel; if 2D array: assume already a spec and flatten."""
    if arr.ndim == 1:
        x = arr.astype(np.float32)
        mx = np.max(np.abs(x))
        if np.isfinite(mx) and mx > 0:
            x = x / mx
        mel = librosa.feature.melspectrogram(y=x, sr=sr, n_fft=1024, hop_length=512, n_mels=n_mels)
        logmel = librosa.power_to_db(mel + 1e-8)
        return logmel.flatten().astype(np.float32)
    elif arr.ndim == 2:
        return arr.astype(np.float32).flatten()
    else:
        raise RuntimeError(f"Unexpected array shape {arr.shape}")


def _load_logmel_subset(paths: Sequence[str], labels: np.ndarray, indices: Sequence[int],
                        *, sr: int = 16000, n_mels: int = 64) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    X, y, used_paths = [], [], []
    for idx in indices:
        p = paths[idx]
        try:
            arr = np.load(p, mmap_mode="r")
            feat = _load_logmel_or_flatten(arr, sr=sr, n_mels=n_mels)
            X.append(feat); y.append(labels[idx]); used_paths.append(p)
        except Exception as e:
            print(f"[WARN] Skipping {p}: {e}")
    return np.asarray(X), np.asarray(y), used_paths


# ========================= Viz: spectrograms for thumbnails =========================

def _ali_spec(x: np.ndarray, fs: int = 16000) -> np.ndarray:
    """Hann spectrogram → dB-like; returns (F,T) slice ~0–1 kHz (adjust as needed)."""
    Lframe2 = 1024
    po = 80
    lov = int(math.ceil((po / 100) * Lframe2))
    taper = get_window('hann', Lframe2)
    Nfft = 2 ** (int(math.floor(math.log2(Lframe2))) + 2)
    f, t, s = spectrogram(x, fs=fs, window=taper, noverlap=lov, nfft=Nfft, mode='magnitude')
    as_max = np.max(s) if np.isfinite(np.max(s)) and np.max(s) > 0 else 1.0
    sdb = 10 * np.log10(np.maximum(s / as_max, 1e-10))
    # keep up to 1000 Hz
    max_inx = np.argmin(np.abs(f - 1000))
    return sdb[:max_inx+1, :]


def _standardize_spec(S: np.ndarray, target_shape=(128, 256)) -> np.ndarray:
    S = np.asarray(S, dtype=np.float32)
    S = (S - np.mean(S)) / (np.std(S) + 1e-8)
    F, T = S.shape
    Ft, Tt = target_shape
    # F
    if F < Ft:
        pt = (Ft - F) // 2
        pb = Ft - F - pt
        S = np.pad(S, ((pt, pb), (0, 0)), mode="constant")
    elif F > Ft:
        s = (F - Ft) // 2
        S = S[s:s+Ft, :]
    # T
    F, T = S.shape
    if T < Tt:
        pl = (Tt - T) // 2
        pr = Tt - T - pl
        S = np.pad(S, ((0, 0), (pl, pr)), mode="constant")
    elif T > Tt:
        s = (T - Tt) // 2
        S = S[:, s:s+Tt]
    return S


def _spec_img_base64(audio_or_spec: np.ndarray, title: str, fs: int = 16000) -> str:
    if audio_or_spec.ndim == 1:
        S = _ali_spec(audio_or_spec.astype(np.float32), fs=fs)
    elif audio_or_spec.ndim == 2:
        S = audio_or_spec.astype(np.float32)
    else:
        return f"<p class='text-danger'>Bad shape: {audio_or_spec.shape}</p>"
    fig, ax = plt.subplots(figsize=(8, 3))
    ax.imshow(S, aspect='auto', origin='lower')
    ax.set_title(title); ax.set_xlabel("Frames"); ax.set_ylabel("Bins")
    fig.tight_layout()
    buf = BytesIO(); fig.savefig(buf, format='png', dpi=120); plt.close(fig); buf.seek(0)
    image_base64 = base64.b64encode(buf.read()).decode("utf-8")
    return f'<img class="d-block w-100" src="data:image/png;base64,{image_base64}" alt="{title}">'


def _load_std_spec_for_index(paths: np.ndarray, idx: int, *, fs: int = 16000,
                             target_shape=(128, 256)) -> Optional[np.ndarray]:
    try:
        arr = np.load(paths[idx], mmap_mode="r")
        if arr.ndim == 1:
            S = _ali_spec(arr.astype(np.float32), fs=fs)
        elif arr.ndim == 2:
            S = arr.astype(np.float32)
        else:
            return None
        return _standardize_spec(S, target_shape=target_shape)
    except Exception:
        return None


def _avg_intra_cluster_spec_cosine(paths: np.ndarray, idxs: np.ndarray, *,
                                   fs: int = 16000, max_samples: int = 50,
                                   target_shape=(128, 256)) -> float:
    if len(idxs) < 2:
        return float("nan")
    if len(idxs) > max_samples:
        rng = np.random.RandomState(42)
        idxs = rng.choice(idxs, size=max_samples, replace=False)
    specs = []
    for i in idxs:
        S = _load_std_spec_for_index(paths, int(i), fs=fs, target_shape=target_shape)
        if S is not None:
            specs.append(S.reshape(-1))
    if len(specs) < 2:
        return float("nan")
    X = np.stack(specs, axis=0)
    sim = cosine_similarity(X)
    iu = np.triu_indices_from(sim, k=1)
    return float(np.mean(sim[iu]))


def _avg_intra_cluster_spec_dtw(paths: np.ndarray, idxs: np.ndarray, *,
                                fs: int = 16000, max_samples: int = 25,
                                target_shape=(128, 256)) -> float:
    try:
        from fastdtw import fastdtw
        from scipy.spatial.distance import euclidean
    except Exception:
        return float("nan")
    if len(idxs) < 2:
        return float("nan")
    if len(idxs) > max_samples:
        rng = np.random.RandomState(42)
        idxs = rng.choice(idxs, size=max_samples, replace=False)
    series = []
    for i in idxs:
        S = _load_std_spec_for_index(paths, int(i), fs=fs, target_shape=target_shape)
        if S is not None:
            series.append(np.mean(S, axis=0))  # collapse freq → time trace
    if len(series) < 2:
        return float("nan")
    n = len(series)
    total, pairs = 0.0, 0
    for a in range(n):
        for b in range(a+1, n):
            d, _ = fastdtw(series[a], series[b], dist=euclidean)
            total += d; pairs += 1
    return float(total / max(pairs, 1))


# ========================= Metrics & clustering =========================

def _hungarian_accuracy(y_true, y_pred) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    true_ids = {t: i for i, t in enumerate(np.unique(y_true))}
    pred_ids = {c: i for i, c in enumerate(np.unique(y_pred))}
    yi = np.vectorize(true_ids.get)(y_true)
    pi = np.vectorize(pred_ids.get)(y_pred)
    W = np.zeros((len(pred_ids), len(true_ids)), dtype=np.int64)
    for i in range(pi.size):
        W[pi[i], yi[i]] += 1
    r, c = linear_sum_assignment(W.max() - W)
    return float(W[r, c].sum() / pi.size)


def _internal_indices(X: np.ndarray, labels: np.ndarray) -> Dict[str, float]:
    out = {"silhouette": float("nan"), "davies_bouldin": float("nan"), "calinski_harabasz": float("nan")}
    uniq = np.unique(labels)
    if len(uniq) <= 1 or len(uniq) >= len(labels):
        return out
    try: out["silhouette"] = float(silhouette_score(X, labels))
    except Exception: pass
    try: out["davies_bouldin"] = float(davies_bouldin_score(X, labels))
    except Exception: pass
    try: out["calinski_harabasz"] = float(calinski_harabasz_score(X, labels))
    except Exception: pass
    return out


def _evaluate_cluster_metrics(embeddings: np.ndarray, idxs: np.ndarray,
                              location_labels: np.ndarray,
                              location_entropy_base: Optional[int] = None) -> Dict[str, float]:
    X = embeddings[idxs]
    if X.shape[0] == 0:
        return {'variance': 0.0, 'mean_sim': 1.0, 'entropy': 0.0, 'quality': 0.0, 'novelty': 0.0}
    center = np.mean(X, axis=0, keepdims=True)
    variance = float(np.mean(np.sum((X - center) ** 2, axis=1)))
    if len(X) > 1:
        cos_sim = cosine_similarity(X)
        iu = np.triu_indices_from(cos_sim, k=1)
        mean_sim = float(np.mean(cos_sim[iu])) if iu[0].size > 0 else 1.0
    else:
        mean_sim = 1.0
    loc_counts = Counter(location_labels[idxs])
    loc_probs = np.array(list(loc_counts.values()), dtype=np.float32)
    loc_probs /= max(loc_probs.sum(), 1e-8)
    base = int(location_entropy_base or len(set(location_labels)))
    loc_entropy = float(entropy(loc_probs, base=base)) if base > 1 else 0.0
    max_ent = np.log2(base) if base > 1 else 1.0
    entropy_score = 1.0 - (loc_entropy / max_ent) if base > 1 else 1.0
    quality = float((mean_sim / (variance + 1e-8)) * entropy_score)
    novelty = float((loc_entropy / max_ent) * variance) if base > 1 else 0.0
    return {'variance': variance, 'mean_sim': mean_sim,
            'entropy': loc_entropy, 'quality': quality, 'novelty': novelty}


# ========================= HTML bits =========================

def _make_carousel(cluster_id: int, scope: str, imgs: List[str]) -> str:
    cid = f"carousel_{scope}_{cluster_id}"
    indicators = "".join(
        f'<button type="button" data-bs-target="#{cid}" data-bs-slide-to="{i}" {"class=active" if i==0 else ""} aria-current="true" aria-label="Slide {i+1}"></button>'
        for i in range(len(imgs))
    )
    items = "".join(
        f'<div class="carousel-item {"active" if i==0 else ""}"><div class="d-flex justify-content-center">{img}</div></div>'
        for i, img in enumerate(imgs)
    )
    return f"""
    <div id="{cid}" class="carousel slide" data-bs-interval="false" data-bs-touch="false">
      <div class="carousel-indicators">{indicators}</div>
      <div class="carousel-inner">{items}</div>
      <button class="carousel-control-prev" type="button" data-bs-target="#{cid}" data-bs-slide="prev">
        <span class="carousel-control-prev-icon" aria-hidden="true"></span>
        <span class="visually-hidden">Previous</span>
      </button>
      <button class="carousel-control-next" type="button" data-bs-target="#{cid}" data-bs-slide="next">
        <span class="carousel-control-next-icon" aria-hidden="true"></span>
        <span class="visually-hidden">Next</span>
      </button>
    </div>
    """


# ========================= Core: analysis → HTML =========================

def analyze_pca_kmeans_to_html(
    *,
    dataset_paths: List[str],
    location_tags: List[str],
    n_components: int = 100,
    random_state: int = 42,
    kmeans_k: Optional[int] = None,
    test_size: float = 0.2,
    test_indices_map: Optional[Dict[str, Sequence[int]]] = None,  # tag -> indices
    do_umap: bool = True,
    umap_components: int = 3,
    samples_per_cluster: int = 4,
    top_n_clusters: Optional[int] = None,
    sr: int = 16000,
    n_mels: int = 64,
) -> str:
    assert len(dataset_paths) == len(location_tags), "dataset_paths and location_tags must match"

    # 1) Build combined TEST set across locations
    X_list, y_list, paths_list, loc_list = [], [], [], []

    for path, tag in zip(dataset_paths, location_tags):
        all_paths, labels = _sorted_paths_and_labels(path)
        all_idx = np.arange(len(all_paths))
        if test_indices_map and tag in test_indices_map:
            test_idx = np.asarray(list(test_indices_map[tag]), dtype=int)
        else:
            _, test_idx = train_test_split(all_idx, test_size=test_size, stratify=labels, random_state=random_state)

        X_test, y_test, used_paths = _load_logmel_subset(all_paths, labels, test_idx, sr=sr, n_mels=n_mels)
        if X_test.size == 0:
            continue

        X_list.append(X_test)
        y_list.append(y_test)
        paths_list.append(np.array(used_paths, dtype=object))
        loc_list.append(np.array([tag] * X_test.shape[0], dtype=object))

    if not X_list:
        raise ValueError("No TEST features loaded from provided datasets.")

    X = np.vstack(X_list).astype(np.float32)
    y = np.concatenate(y_list, axis=0)
    original_paths = np.concatenate(paths_list, axis=0)
    location_labels = np.concatenate(loc_list, axis=0).astype(object)

    # 2) Scale + PCA (fit on TEST only)
    t0 = time.perf_counter()
    scaler = StandardScaler().fit(X)
    Xs = scaler.transform(X)
    pca = PCA(n_components=n_components, random_state=random_state).fit(Xs)
    Z = pca.transform(Xs)  # TEST embeddings
    print(f"[INFO] PCA on TEST: {Z.shape} • {(time.perf_counter()-t0):.2f}s")

    # 3) KMeans on TEST
    k = int(kmeans_k) if kmeans_k is not None else len(np.unique(y))
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=random_state)
    cluster_labels = kmeans.fit_predict(Z)

    # 4) Global metrics
    uniq = np.unique(cluster_labels)
    valid = (len(uniq) > 1) and (Z.shape[0] > len(uniq))

    def _safe(fn, *args):
        try:
            return float(fn(*args)) if valid else float("nan")
        except Exception:
            return float("nan")

    sil = _safe(silhouette_score, Z, cluster_labels)
    dbi = _safe(davies_bouldin_score, Z, cluster_labels)
    ch  = _safe(calinski_harabasz_score, Z, cluster_labels)

    # external
    def _to_float(v): 
        try: return float(v)
        except: return float("nan")

    try:
        ari  = _to_float(adjusted_rand_score(y, cluster_labels)) if valid else float("nan")
    except Exception:
        ari = float("nan")
    try:
        ami  = _to_float(adjusted_mutual_info_score(y, cluster_labels)) if valid else float("nan")
    except Exception:
        ami = float("nan")
    try:
        hacc = _to_float(_hungarian_accuracy(y, cluster_labels)) if valid else float("nan")
    except Exception:
        hacc = float("nan")

    # 5) UMAP (on TEST embeddings)
    if do_umap:
        reducer = umap.UMAP(n_components=umap_components, random_state=random_state)
        U = reducer.fit_transform(Z)
        if umap_components == 3:
            fig = px.scatter_3d(
                x=U[:,0], y=U[:,1], z=U[:,2],
                #color=[str(c) for c in cluster_labels],
                color=y.astype(str),
                symbol=location_labels,
                hover_data={"Cluster": cluster_labels, "Class": y.astype(str)},
                title=(f"PCA+KMeans TEST (k={k}) | "
                       f"Sil {sil:.3f} · DBI {dbi:.3f} · CH {ch:.1f} · "
                       f"ARI {ari:.3f} · AMI {ami:.3f} · H-Acc {hacc:.3f}"),
                opacity=0.85, height=800
            )
        else:
            fig = px.scatter(
                x=U[:,0], y=U[:,1],
                #color=[str(c) for c in cluster_labels],
                color=y.astype(str),
                symbol=location_labels,
                hover_data={"Cluster": cluster_labels, "Class": y.astype(str)},
                title=(f"PCA+KMeans TEST (k={k}) | "
                       f"Sil {sil:.3f} · DBI {dbi:.3f} · CH {ch:.1f} · "
                       f"ARI {ari:.3f} · AMI {ami:.3f} · H-Acc {hacc:.3f}"),
                opacity=0.85, height=700
            )
        umap_html = pio.to_html(fig, include_plotlyjs="cdn", full_html=False)
    else:
        umap_html = "<p class='text-muted'>UMAP disabled.</p>"

    # 6) Per-cluster cards
    base_for_entropy = len(set(location_labels))
    cluster_ids = sorted(set(cluster_labels), key=lambda c: np.sum(cluster_labels==c), reverse=True)
    if top_n_clusters is not None:
        cluster_ids = cluster_ids[:int(top_n_clusters)]

    spec_cos_scores, spec_dtw_scores = [], []
    blocks = []

    for cid in cluster_ids:
        idxs = np.where(cluster_labels == cid)[0]
        if idxs.size == 0:
            continue

        sz = len(idxs)
        loc_counts = Counter(location_labels[idxs])
        cls_counts = Counter(y[idxs])

        metrics = _evaluate_cluster_metrics(Z, idxs, location_labels, location_entropy_base=base_for_entropy)
        spec_cos = _avg_intra_cluster_spec_cosine(original_paths, idxs, fs=sr, max_samples=50)
        spec_dtw = _avg_intra_cluster_spec_dtw(original_paths, idxs, fs=sr, max_samples=25)

        if np.isfinite(spec_cos): spec_cos_scores.append(spec_cos)
        if np.isfinite(spec_dtw): spec_dtw_scores.append(spec_dtw)

        meta_html = "<p><strong>Location Distribution:</strong></p><ul>" + "".join(
            f"<li><b>{loc}</b>: {count} ({count/sz:.1%})</li>" for loc, count in loc_counts.items()
        ) + "</ul>"
        meta_html += "<p><strong>Class Distribution:</strong></p><ul>" + "".join(
            f"<li>{str(cls)}: {count}</li>" for cls, count in cls_counts.items()
        ) + "</ul>"

        meta_html += f"""
        <p><strong>Cluster Metrics (Embedding Space):</strong></p>
        <ul>
          <li>Size: {sz}</li>
          <li>Intra-Cluster Variance: {metrics['variance']:.4f}</li>
          <li>Mean Cosine Similarity (Embeddings): {metrics['mean_sim']:.4f}</li>
          <li>Location Entropy: {metrics['entropy']:.3f}</li>
          <li>Composite Quality Score: {metrics['quality']:.4f}</li>
          <li><strong>Novelty Score:</strong> {metrics['novelty']:.4f}</li>
        </ul>
        <p><strong>Spectrogram Consistency (Signal Space):</strong></p>
        <ul>
          <li>Mean Spectrogram Cosine Similarity (↑ better): {spec_cos if np.isfinite(spec_cos) else float('nan'):.4f}</li>
          <li>Mean Spectrogram DTW Distance (↓ better): {spec_dtw if np.isfinite(spec_dtw) else float('nan'):.2f}</li>
        </ul>
        """

        # Nearest-to-center exemplars
        center = np.mean(Z[idxs], axis=0, keepdims=True)
        dists = np.linalg.norm(Z[idxs] - center, axis=1)
        order = np.argsort(dists)
        pick = idxs[order[:min(samples_per_cluster, len(order))]]

        thumbs = []
        for i, p in enumerate(pick):
            try:
                arr = np.load(original_paths[p], mmap_mode="r")
                title = f"#{i+1} | {location_labels[p]} | Class {str(y[p])}"
                thumbs.append(_spec_img_base64(arr, title, fs=sr))
            except Exception as e:
                thumbs.append(f"<p class='text-danger'>Error: {e}</p>")

        blocks.append(f"<div class='col-md-6 mb-4'><h4>Cluster {cid}</h4>{meta_html}{_make_carousel(cid, 'pca', thumbs)}</div>")

    cluster_html = ""
    for i in range(0, len(blocks), 2):
        cluster_html += "<div class='row'>" + "".join(blocks[i:i+2]) + "</div>"

    # 7) Global spectrogram-consistency summary
    global_spec_cos = float(np.mean(spec_cos_scores)) if len(spec_cos_scores) else float("nan")
    global_spec_dtw = float(np.mean(spec_dtw_scores)) if len(spec_dtw_scores) else float("nan")

    summary_card = f"""
    <div class='row mb-4'>
      <div class='col-md-12'>
        <div class='alert alert-info' role='alert'>
          <h5 class='mb-2'>Spectrogram Consistency (Cluster Averages)</h5>
          <ul class='mb-0'>
            <li><strong>Mean Spectrogram Cosine Similarity</strong>: {global_spec_cos if np.isfinite(global_spec_cos) else float('nan'):.4f} (↑ better)</li>
            <li><strong>Mean Spectrogram DTW Distance</strong>: {global_spec_dtw if np.isfinite(global_spec_dtw) else float('nan'):.2f} (↓ better)</li>
          </ul>
          <small>Cosine computed on standardized spectrograms (z-norm; padded/cropped). DTW optional and subsampled.</small>
        </div>
      </div>
    </div>
    """

    # Assemble section
    title_txt = (f"PCA+KMeans TEST (k={k}) • "
                 f"Sil={sil:.3f} | DBI={dbi:.3f} | CH={ch:.1f} | "
                 f"ARI={ari:.3f} | AMI={ami:.3f} | H-Acc={hacc:.3f}")

    return f"""
    <div class='section'>
      <h2>{title_txt}</h2>
      <div class="container">
        <div class="row justify-content-center mb-4">
          <div class="col-md-12 d-flex justify-content-center">{umap_html}</div>
        </div>
        {summary_card}
      </div>
      {cluster_html}
    </div>
    """


def generate_pca_kmeans_full_report(
    *,
    dataset_paths: List[str],
    location_tags: List[str],
    out_html: str = "pca_kmeans_unsupervised_report.html",
    n_components: int = 100,
    random_state: int = 42,
    kmeans_k: Optional[int] = None,
    test_size: float = 0.2,
    test_indices_map: Optional[Dict[str, Sequence[int]]] = None,  # tag -> indices
    do_umap: bool = True,
    umap_components: int = 3,
    samples_per_cluster: int = 4,
    top_n_clusters: Optional[int] = None,
    sr: int = 16000,
    n_mels: int = 64,
) -> str:
    section = analyze_pca_kmeans_to_html(
        dataset_paths=list(dataset_paths),
        location_tags=list(location_tags),
        n_components=n_components,
        random_state=random_state,
        kmeans_k=kmeans_k,
        test_size=test_size,
        test_indices_map=test_indices_map,
        do_umap=do_umap,
        umap_components=umap_components,
        samples_per_cluster=samples_per_cluster,
        top_n_clusters=top_n_clusters,
        sr=sr,
        n_mels=n_mels,
    )
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Unsupervised Clustering Report (PCA + KMeans • TEST-ONLY)</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ font-family: Arial, sans-serif; padding: 20px; background-color: #f5f5f5; }}
            h1 {{ color: #2c3e50; }}
            h2, h4 {{ color: #34495e; }}
            hr {{ border-top: 2px solid #bbb; margin-top: 40px; margin-bottom: 40px; }}
            .section {{ margin-bottom: 60px; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
        </style>
    </head>
    <body>
      <h1 class='mb-4'>Unsupervised Latent Clustering Report (PCA + KMeans • TEST-ONLY)</h1>
      <p><strong>Locations:</strong> {', '.join(location_tags)}</p>
      <p><strong>TEST split:</strong> {int(test_size*100)}% (stratified) • Random state: {random_state}</p>
      <hr>
      {section}
    </body></html>
    """
    with open(out_html, "w", encoding="utf-8") as f:
        f.write(html)
    print(f"✅ Report saved to: {out_html}")
    return out_html


# ------------------------ Example direct call ------------------------
if __name__ == "__main__":
    # Single site example (adjust paths + tags)
    report_path = generate_pca_kmeans_full_report(
        dataset_paths=["/notebooks/dataset_preprocessed"],
        location_tags=["PR_U1137"],
        out_html="pca_k6.html",
        n_components=100,
        random_state=42,
        kmeans_k=60,
        test_size=0.2,
        test_indices_map=None,     # or {"PR_U1137": [list_of_test_indices]}
        do_umap=True,
        umap_components=3,
        samples_per_cluster=10,
        top_n_clusters=None,
        sr=16000,
        n_mels=64,
    )
    print("Report:", report_path)

[INFO] PCA on TEST: (82655, 100) • 49.04s


/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



✅ Report saved to: pca_k6.html
Report: pca_k6.html
